# <center>Big Data for Engineers &ndash; Exercises</center>
## <center>Spring 2021 &ndash; Week 10 &ndash; ETH Zurich</center>
## <center>Spark Dataframes and SparkSQL</center>

## Introduction
For this exercise please create a Spark cluster on Azure as in the previous exercises. 
- If you have performance problems, check the yarn UI (```https://<cluster_name>.azurehdinsight.net/yarnui/hn/cluster```) and make sure that there are no unwanted applications hogging the resources.

## Getting the data


- Log in using ssh to your cluster:  ```ssh <ssh_user_name>@<cluster_name>-ssh.azurehdinsight.net```
- Download the data: ```wget https://bigdata2020exassets.blob.core.windows.net/ex09/orders.jsonl```
- Upload the data to HDFS: ```hdfs dfs -put orders.jsonl /tmp/```


After you have uploaded the dataset into the Azure Blob, upload this notebook onto the Spark Jupyter server  (`https://<cluster-name>.azurehdinsight.net/jupyter`).

## Spark Session
When you execute the first cell in this notebook, a spark session will be created automatically, you can print information about the session with the ```%%info``` magics

In [ ]:
print("Hello")

In [ ]:
%%info

By default the session is created with 3 Spark executors. We can change the configuration with the ```%%configure``` magics. Make sure you have enough vCores/Memory. You can see your total amount of available resources in yarnUI. 

In [ ]:
%%configure -f
{"executorMemory": "2G", "numExecutors": 6, "driverMemory": "4G"}

## 1. Spark Dataframes

Spark Dataframes allow the user to perform simple and efficient operations on data, as long as the data is structured and has a schema. Dataframes are similar to relational tables in relational databases: conceptually a dataframe is a specialization of a Spark RDD with schema information attached. You can find more information in Karau, H. et al. (2015). Learning Spark, Chapter 9 (optional reading).

### 1.1. Data preprocessing

In [ ]:
import json

path = "/tmp/orders.jsonl"
orders_df = spark.read.json(path).cache()

The type of our dataset object is DataFrame

In [ ]:
type(orders_df)

Print the schema

In [ ]:
orders_df.printSchema()

Print one row

In [ ]:
orders_df.limit(1).collect()

You can access the underlying RDD object and use any functions you learned for Spark RDDs.

In [ ]:
orders_df.rdd.filter(lambda ordr: ordr.customer.last_name == "Landry").count()

### 1.2. Dataframe Operations
We perform some queries using operations on Dataframes ([Here](https://spark.apache.org/docs/2.3.0/sql-programming-guide.html#untyped-dataset-operations-aka-dataframe-operations) is a guide on DF Operations with a link to the [API Documentation](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html))

We can select columns and show the result

In [ ]:
orders_df.select("customer.first_name", "customer.last_name").limit(5).show()

As you can see we can navigate to the nested items with the dot

In [ ]:
orders_df.filter(orders_df["customer.last_name"] == "Landry").count()

How about nested arrays?

In [ ]:
orders_df.select("order_id", "items").orderBy("order_id").limit(5).show()

Let us try to find orders of a fan.

In [ ]:
orders_df.filter(orders_df["items.product"] == "fan").count()

The above code doesn't work! Use ```array contains``` instead.

In [ ]:
from pyspark.sql.functions import array_contains

orders_df.filter(array_contains("items.product", "fan")).count()

Let us try to unnest the data.

Unnest the products with explode.

Explode will generate as many rows as there are elements in the array and match them to other attributes.

In [ ]:
from pyspark.sql.functions import explode

orders_df.select(explode("items").alias("i"), "i.product", "order_id").orderBy("order_id").limit(5).show()

Now we can use this table to filter.

In [ ]:
exploded_df = orders_df.select(explode("items").alias("i"), "i.product", "order_id")
exploded_df.filter(exploded_df["product"] == "fan").count()

You might have tried to access the i.product column directly using a ```.filter``` right after the ```.select```. That, however, does not work, because the column is not available to ```orders_df``` when creating a clause like ```(orders_df["i.product"] == "fan")```. A possible workaround when using Dataframe operations is that of using a string clause in ```.filter```, so that the product column will be resolved after it has been added with the ```.select```.

In [ ]:
orders_df.select(explode("items").alias("i"), "i.product", "order_id").filter("product = 'fan'").count()

Project the nested columns

In [ ]:
orders_df.select(explode("items").alias("i"), "*").select(
    "order_id", "customer.*", "date", "i.*").limit(3).show()

### 1.3 Exercises

1) Find the average quantity at which each product is purchased. Only show the top 10 products by quantity. (Hint: you may need to import the function ```desc``` from ```pyspark.sql.functions``` to define descending order)

In [ ]:
# Your Code Here

2) Find the most expensive order

In [ ]:
# Your Code Here

## 2. Spark SQL

Spark SQL allows the users to formulate their queries using SQL. The requirement is the use of Dataframes, which as said before are similar to relational tables. In addition to a familiar interface, writing queries in SQL might provide better performance than RDDs, inheriting efficiency from the Dataframe operations, while also performing automatic optimization of queries.

In order to use sql we need to create a temporary table.

This table only exists for the current session.

In [ ]:
orders_df.registerTempTable("orders")

### 2.1 Queries

Finally, run SQL queries on the registered tables. We will run the same queries as during the previous section, but with SQL.

As you can see we can navigate to the nested items with the dot.

In [ ]:
%%sql
-- Finally, run SQL queries on the registered tables
-- As you can see we can navigate to the nested items with the dot
SELECT count(*)
FROM orders
WHERE orders.customer.last_name == "Landry"

How about nested arrays?

In [ ]:
%%sql
-- How about nested arrays?
SELECT order_id, items
FROM orders AS o
ORDER BY order_id
LIMIT 5

Let us try to find orders of a fan.

In [ ]:
%%sql 
SELECT count(*)
FROM orders
WHERE items.product = "fan"

The above code doesn't work! Use ```array contains``` instead.

In [ ]:
%%sql

SELECT count(*)
FROM orders
WHERE array_contains(items.product, "fan")

Let us try to unnest the data.

Unnest the products with explode.

Explode will generate as many rows as there are elements in the array and match them to other attributes.

In [ ]:
%%sql
SELECT explode(items) as i, i.product, order_id
FROM orders
ORDER BY order_id
limit 5

Now we can use this table to filter.

In [ ]:
%%sql
-- Filter on product
SELECT count(*)
    FROM (
    SELECT explode(items) as i, i.product, order_id
    FROM orders
    ORDER BY order_id
    )
WHERE product = "fan"

You might have tried to access the i.product column directly in the same ```SELECT``` clause. That, however, does not work, because the column is not available to the ```WHERE``` clause. In order to access the built columns directly, we need to unnest the data and make it part of our ```FROM``` clause. ```LATERAL VIEW``` lets us do just that, matching each non-array attribute to an unnested row from the array.  

In [ ]:
%%sql
SELECT *
FROM orders lateral view explode(items) as flat_items
WHERE flat_items.product = "fan"
ORDER BY order_id
LIMIT 3

Project the nested columns

In [ ]:
%%sql
SELECT order_id, customer.first_name, customer.last_name, date, flat_items.*
FROM orders lateral view explode(items) item_table as flat_items
WHERE flat_items.product = "fan"
ORDER BY order_id
LIMIT 3

Having built an unnested table, we can now easily aggregate over the previously nested columns

### 2.2 Exercises

1) Find the average quantity at which each product is purchased. Only show the top 10 products by quantity. 

In [ ]:
%%sql
# Your SQL Query Here

2) Find the most expensive order

In [ ]:
%%sql
# Your SQL Query Here

## 3. Create Nestedness

We've already had a look at the solution of dataframes/SparkSQL towards unnesting arrays by using <font face="courier">explode</font> method. For the other way round, Spark Dataframes / Spark SQL also provide ways for us to nest our data by creating arrays, especially after clauses like <font face="courier">group by</font>.

In traditional PostgreSQL, we have to use one of the aggregation functions (<font face="courier">max, sum, count,</font> ...) to process the result after the <font face="courier">group by</font> operation. For example, for each customer (assume there are no customers with both the same first name and last name), we want to find out how many dates at which they placed an order. The query should look like this (fill in the queries yourself):

In [ ]:
# Your Code Here

In [ ]:
%%sql
# Your SQL Query Here

But what if we are interested not only in the number of dates, but the actual
dates themselves? Luckily Spark Dataframes / Spark SQL do provide us with methods to preserve the original information of the date list. If now we would like to know for each customer, on which dates they placed an order, we shall use <font face="courier">collect_set</font> method:

In [ ]:
from pyspark.sql.functions import collect_set
orders_df.groupBy("customer.first_name", "customer.last_name").agg(collect_set("date")).show()

In [ ]:
%%sql
select customer.first_name, customer.last_name, collect_set("date") from orders group by customer.first_name, customer.last_name

For some other cases where we want to preserve all the entries rather than the de-duplicated ones, we can use  <font face="courier">collect_list</font> method. For example, for each date we want to record the first and last names of customers. Since two customers might share the same last name, we need to collect all of them. The query should look like this:

In [ ]:
from pyspark.sql.functions import collect_list
orders_df.groupBy("date").agg(collect_list("customer.last_name")).show()

In [ ]:
%%sql
select "date", collect_list(customer.last_name) from orders group by "date"

Now try it on yourself.

For every order in 2016-1-1, return the list of products that appeared in that order:

In [ ]:
# Your Code Here

For every product, return the set of dates at which it's purchased:

In [ ]:
# Your Code Here

One of the drawbacks of the <font face="courier">collect_set/collect_list</font> method is they only accept one column as the argument. Later we will see how we can create nestedness on pretty much everything after we get the hang of the mighty JSONiq.